# PREPARE DATA FOR FRAME EXTRACTION

In [7]:
import os
import json

def process_json_files(root_folder):
    """
    Scans root_folder for JSON files containing annotations, aggregates them by clip_uid into dict mapping clip to frame
    and to annot_uid

    Args:
    - root_folder (str): Path to the annots folder

    Returns:
    - dict: A dictionary mapping clip UIDs to their frames' annotations, categorized by frame number and type ('pre', 'pnr', 'post'), 
      alongside the annot_uid.
    """
    result_dict = {}
    for subdir, _, files in os.walk(root_folder):
        for file in files:
            if file.endswith(".json"):
                annot_uid = os.path.basename(subdir)  # assuming annot_uid is the subfolder name
                file_path = os.path.join(subdir, file)
                
                with open(file_path, 'r') as json_file:
                    data = json.load(json_file)
                    
                    clip_uid = data.get('clip_uid')
                    pre_frame_number = data.get('pre_frame')['clip_frame_number']
                    pnr_frame_number = data.get('pnr_frame')['clip_frame_number']
                    post_frame_number = data.get('post_frame')['clip_frame_number']
                    
                    if clip_uid:
                        if clip_uid not in result_dict:
                            result_dict[clip_uid] = {}
                        
                        frames = [(pre_frame_number, 'pre'), (pnr_frame_number, 'pnr'), (post_frame_number, 'post')]
                        
                        for frame_number, frame_type in frames:
                            if frame_number:
                                if frame_number not in result_dict[clip_uid]:
                                    result_dict[clip_uid][frame_number] = []
                                
                                result_dict[clip_uid][frame_number].append((frame_type, annot_uid))
                            
    return result_dict

# Modify this to the path of your "annotations_all" folder
root_folder = "annotations_all"
processed_data = process_json_files(root_folder)

In [3]:
processed_data['e9be1118-a5cf-4431-b2e8-e3edcfa9f949']

{48: [('pre',
   '2ce11f8f93edca540164c11adcba4bcce0fdebf378af50757a688f0abc13e0cc')],
 62: [('pnr',
   '2ce11f8f93edca540164c11adcba4bcce0fdebf378af50757a688f0abc13e0cc')],
 69: [('post',
   '2ce11f8f93edca540164c11adcba4bcce0fdebf378af50757a688f0abc13e0cc')],
 72: [('pre',
   '77db171652757f04902cbe8aa2422a97c62f3ec1ee0799bb9b021877c109eaed')],
 75: [('pnr',
   '77db171652757f04902cbe8aa2422a97c62f3ec1ee0799bb9b021877c109eaed')],
 102: [('post',
   '77db171652757f04902cbe8aa2422a97c62f3ec1ee0799bb9b021877c109eaed')],
 136: [('pre',
   '50432b97708db6fa1d74b9a9d873756402ac489d911364339a2741082011ac87')],
 141: [('pnr',
   '50432b97708db6fa1d74b9a9d873756402ac489d911364339a2741082011ac87')],
 161: [('post',
   '50432b97708db6fa1d74b9a9d873756402ac489d911364339a2741082011ac87')],
 164: [('pre',
   'db01246efd46f56d4718037a02a63f64b3c05b52d03ddffde3c8cc9bbace0576')],
 167: [('pnr',
   'db01246efd46f56d4718037a02a63f64b3c05b52d03ddffde3c8cc9bbace0576')],
 185: [('post',
   'db01246efd46f5

# EXTRACT FRAMES

In [8]:
import cv2
import os
from tqdm import tqdm

def extract_frames(video_folder, output_folder, data):
    """
    Extracts frames from clip files based on provided data and saves them. Iterates through each video clip
    identified by clip_uid and extracts frames specified in the `data` parameter.
    
    Args:
    - video_folder (str): Path to the folder containing the Ego4D clips.
    - output_folder (str): Path to the folder where extracted frames will be saved.
    - data (dict): A dictionary retrieved with process_json_files() function.
    """
    CNT_CLIP = 0
    for clip_uid, frames_data in data.items():
        print('Processing clip {}'.format(CNT_CLIP))
        CNT_CLIP+=1
        video_path = os.path.join(video_folder, f"{clip_uid}.mp4")
        cap = cv2.VideoCapture(video_path)

        if not cap.isOpened():
            print(f"Error: Couldn't open the video file {video_path}")
            continue

        for frame_number, annotations in tqdm(frames_data.items()):
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number - 1)  # setting the frame position

            ret, frame = cap.read()
            if ret:
                for frame_type, annot_uid in annotations:
                    output_path = os.path.join(output_folder, annot_uid, f"{clip_uid}_{frame_type}.jpg")
                    os.makedirs(os.path.dirname(output_path), exist_ok=True)
                    cv2.imwrite(output_path, frame)
            else:
                print(f"Error: Couldn't read frame {frame_number} from video {video_path}")

        cap.release()

# Modify these paths
video_folder = ...
output_folder = "frames_all"

extract_frames(video_folder, output_folder, processed_data)

  0%|          | 0/150 [00:00<?, ?it/s]

Processing clip 0


  0%|          | 0/81 [00:00<?, ?it/s]

Processing clip 1


  0%|          | 0/21 [00:00<?, ?it/s]

...

  0%|          | 0/21 [00:00<?, ?it/s]

Processing clip 219


  0%|          | 0/80 [00:00<?, ?it/s]

Processing clip 220


100%|██████████| 80/80 [00:27<00:00,  2.86it/s]


In [9]:
len(os.listdir('frames_all'))

6250

# ADD BOTH HANDS BBOX

In [8]:
import json

def compute_union_bbox(bbox1, bbox2):
    """
    Computes union of 2 bboxes, used for both_hands
    """
    x_min = min(bbox1['x'], bbox2['x'])
    y_min = min(bbox1['y'], bbox2['y'])
    
    x_max = max(bbox1['x'] + bbox1['width'], bbox2['x'] + bbox2['width'])
    y_max = max(bbox1['y'] + bbox1['height'], bbox2['y'] + bbox2['height'])
    
    return {
        'x': x_min,
        'y': y_min,
        'width': x_max - x_min,
        'height': y_max - y_min
    }

def add_union_bbox(data):
    """
    Adds both_hands bounding box to the annotation
    """
    for key in ['pre_frame', 'pnr_frame', 'post_frame']:
        left_hand = None
        right_hand = None
        for annotation in data[key]['bbox']:
            if annotation['object_type'] == 'left_hand':
                left_hand = annotation['bbox']
            elif annotation['object_type'] == 'right_hand':
                right_hand = annotation['bbox']
        
        if left_hand and right_hand:
            union_bbox = compute_union_bbox(left_hand, right_hand)
        elif left_hand:
            union_bbox = left_hand
        elif right_hand:
            union_bbox = right_hand
        else:
            continue
        
        data[key]['bbox'].append({
            "object_type": "both_hands",
            "structured_noun": None,
            "instance_number": 1,
            "bbox": union_bbox
        })

    return data

def read_json(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

def write_json(filename, data):
    with open(filename, 'w') as file:
        json.dump(data, file, indent=4)

def modify_json_files(root_folder):
    """
    Function to enrich the existing annotations with both_hands bounding boxes
    """
    for subdir, _, files in os.walk(root_folder):
        for file in files:
            if file.endswith(".json"):
                file_path = os.path.join(subdir, file)
                
                # Reading the JSON file
                with open(file_path, 'r') as json_file:
                    data = json.load(json_file)
                
                # Modifying the data with your function
                add_union_bbox(data)
                
                # Writing the modified data back to the file
                with open(file_path, 'w') as json_file:
                    json.dump(data, json_file, indent=4)

In [13]:
modify_json_files('annotations_all')

# CREATE MANIFEST

Creating the list of data objects to be annotated

In [18]:
JOB_NAME = 'jobs_all'
MANIFEST_NAME = '_unict_all.manifest'
BUCKET_NAME = ...
ANNOTS_PATH = 'annotations_all'

folders = os.listdir('frames_all')
ls_manifest = []
k = "source-ref"
try_id = 1
for annot_uid in folders:
    v = "s3://{}/data/jobs/{}/{}/{}-sequence-{}.json".format(BUCKET_NAME, JOB_NAME, annot_uid, annot_uid, try_id)
    ls_manifest.append({k:v})
cnt = 0
with open(MANIFEST_NAME, 'w') as f:
    for item in ls_manifest:
        json.dump(item, f)
        f.write('\n')
        cnt+=1
print('Total number of SG: {}'.format(cnt))

Total number of SG: 6250


# CREATE JOBS

In [24]:
for ii, annot_id in enumerate(folders):
    file_list = os.listdir(os.path.join(ANNOTS_PATH, annot_id))
    clip_id = os.path.basename([file for file in file_list if file.endswith('.json')][0])
    data = {"seq-no":ii+1,
           "prefix":"s3://{}/data/jobs/{}/{}/".format(BUCKET_NAME, JOB_NAME,annot_id),
           "number-of-frames":3,
           "frames":[{"frame-no":0,"frame":"{}_pnr.jpg".format(clip_id[:-5])},
            {"frame-no":1,"frame":"{}_post.jpg".format(clip_id[:-5])},
            {"frame-no":2,"frame":"{}_pre.jpg".format(clip_id[:-5])}]}
    os.makedirs('{}/{}'.format(JOB_PATH,annot_id), exist_ok=True)
    file_to_save = '{}/{}/{}-sequence-{}.json'.format(JOB_PATH, annot_id, annot_id, try_id)
    with open(file_to_save, 'w') as f:
        json.dump(data, f)

# Create Narration-Annotation mapping

In [48]:
mappings = []
for folder in os.listdir('annotations_all'):
    for file in os.listdir(os.path.join('annotations_all',folder)):
        if file.endswith('.json'):
            pf = os.path.join('annotations_all',folder,file)
            with open(pf,'r') as pf_json:
                cont = json.load(pf_json)['narrations_infos'][0]
            mappings.append({folder:cont})
print(len(mappings))
narr_annot_mapping_new = {'mappings': mappings}
with open('narration_annotation_mappings_unict_all.json', 'w') as outfile:
    json.dump(narr_annot_mapping_new, outfile)

6250
